# 05 - Results Visualization

This notebook generates publication-quality figures and tables.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.data.synthetic_data import SyntheticDataGenerator
from src.portfolio.baseline import BaselinePortfolio
from src.portfolio.state_conditioned import StateConditionedPortfolio
from src.visualization.figures import FigureGenerator
from src.visualization.tables import TableGenerator
from src.visualization.styles import set_publication_style, PlotStyles

set_publication_style()
np.random.seed(42)

In [ ]:
# Load data
generator = SyntheticDataGenerator(seed=42)
data = generator.generate(n_months=732)

factors = data['factors']
volatility = data['volatility']
regimes = data['regimes']
if isinstance(regimes, pd.DataFrame):
    regimes = regimes['regime']

## 1. Generate All Figures

In [ ]:
# Initialize figure generator
fig_gen = FigureGenerator(output_dir='../output/figures', formats=['pdf', 'png'])

# Prepare data
fig_data = {
    'factors': factors,
    'regimes': regimes,
    'states': volatility,
    'volatility': volatility,
    'ic': data.get('ic'),
}

In [ ]:
# Figure 1: State Space
fig = fig_gen.figure1_state_space(volatility, regimes, save=False)
plt.show()

In [ ]:
# Figure 2: Cumulative Performance
fig = fig_gen.figure2_cumulative_performance(factors, regimes, save=False)
plt.show()

In [ ]:
# Figure 4: Factor Returns by State
fig = fig_gen.figure4_factor_returns_by_state(factors, regimes, save=False)
plt.show()

In [ ]:
# Figure 6: Drawdown Comparison
fig = fig_gen.figure6_drawdown(factors['Momentum'], regimes, save=False)
plt.show()

In [ ]:
# Figure 8: Volatility Ratio Distribution
fig = fig_gen.figure8_volatility_ratio_distribution(volatility['rho_sigma'], regimes, save=False)
plt.show()

## 2. Generate All Tables

In [ ]:
# Initialize table generator
table_gen = TableGenerator(output_dir='../output/tables')

In [ ]:
# Table 1: Summary Statistics
latex = table_gen.table1_summary_statistics(factors, regimes, save=False)
print(latex)

In [ ]:
# Table 2: State-Conditional Returns
latex = table_gen.table2_state_conditional_returns(factors, regimes, save=False)
print(latex)

In [ ]:
# Table 3: Regime Frequencies
latex = table_gen.table3_regime_frequencies(regimes, save=False)
print(latex)

## 3. Save All Outputs

In [ ]:
# Generate and save all figures
fig_gen.generate_all_figures(fig_data)
print("All figures saved!")

In [ ]:
# Generate and save all tables
table_gen.generate_all_tables(fig_data)
print("All tables saved!")

## 4. Custom Visualizations

In [ ]:
# Create a summary dashboard
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Panel 1: State space
ax1 = axes[0, 0]
for regime in PlotStyles.REGIME_ORDER:
    mask = regimes == regime
    if mask.sum() > 0:
        ax1.scatter(
            volatility.loc[mask, 'sigma_1m'] * 100,
            volatility.loc[mask, 'rho_sigma'],
            c=PlotStyles.get_regime_color(regime),
            label=regime, alpha=0.5, s=15
        )
ax1.set_xlabel('Volatility (%)')
ax1.set_ylabel('Vol Ratio')
ax1.set_title('State Space')
ax1.legend(fontsize=8)

# Panel 2: Factor returns by regime
ax2 = axes[0, 1]
regime_means = pd.DataFrame()
for regime in PlotStyles.REGIME_ORDER:
    mask = regimes == regime
    if mask.sum() > 0:
        regime_means[regime] = factors.loc[mask].mean() * 100
regime_means.T.plot(kind='bar', ax=ax2, color=[PlotStyles.get_factor_color(f) for f in factors.columns])
ax2.axhline(y=0, color='black', linewidth=0.5)
ax2.set_ylabel('Mean Return (%)')
ax2.set_title('Returns by Regime')
ax2.legend(fontsize=8)
ax2.tick_params(axis='x', rotation=45)

# Panel 3: Cumulative returns
ax3 = axes[1, 0]
for col in factors.columns:
    cum = np.cumsum(factors[col]) * 100
    ax3.plot(cum.index, cum.values, label=col, 
             color=PlotStyles.get_factor_color(col), linewidth=1.5)
ax3.set_xlabel('Date')
ax3.set_ylabel('Cumulative Return (%)')
ax3.set_title('Cumulative Performance')
ax3.legend(fontsize=8)

# Panel 4: Regime frequencies
ax4 = axes[1, 1]
freq = regimes.value_counts()
colors = [PlotStyles.get_regime_color(r) for r in freq.index]
ax4.pie(freq.values, labels=freq.index, colors=colors, autopct='%1.1f%%')
ax4.set_title('Regime Distribution')

plt.suptitle('Volatility Path States: Summary Dashboard', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Summary

This notebook has generated:
- 8 publication-quality figures
- 5 LaTeX tables
- Custom summary visualizations

All outputs are saved to the `output/` directory.